In [176]:
from typing_extensions import TYPE_CHECKING  # type: ignore

if TYPE_CHECKING:
    from rp2.asm_pio import *

In [177]:
# %%micropython --reset
from rp2 import PIO

# remove all programs from both PIOs
for n in (0, 1):
    pio = PIO(n)
    pio.remove_program()

True


In [178]:
# %%micropython
from machine import Pin, Signal

sig_dmx_send = Signal(Pin(13, Pin.OUT, Pin.PULL_DOWN))

sig_dmx_send.off()

In [179]:
# %%micropython
# Example using PIO to create a UART TX interface

from machine import Pin, Signal
from rp2 import PIO, StateMachine, asm_pio

# -----------------------------------------------
# Wiring schema for the DMX TX
p1 = Pin(1, Pin.OUT, value=0)  
pin_dmx_tx = Pin(15, Pin.OUT)
pin_dmx_rx = Pin(14, Pin.IN, pull=Pin.PULL_DOWN)
sig_dmx_send = Signal(Pin(13, Pin.OUT, Pin.PULL_DOWN))

# sig_dmx_send.on()


# Minimum universe length is ~5 channels / slots


@asm_pio(
    set_init=PIO.OUT_LOW,
    sideset_init=PIO.OUT_LOW,
    out_init=PIO.OUT_LOW,
    # autopull=True,  # ?
    out_shiftdir=PIO.SHIFT_RIGHT,  # ?
    push_thresh=8,
)
# fmt: off
def dmx_send():
    BREAK_LOOPS = 23
    # Assert break condition
    # Sender DMX break signal is 92us >, so we need to loop at least 92 / 8 = 12 times
    # TODO: Actually should be at least 92us
    set(x, 21)      .side(0)                # BREAK condition for 176us

    label("breakloop")                      # Loop X times, each loop iteration is 8 cycles.
    jmp(x_dec, "breakloop")             [7] # Each loop iteration is 8 cycles.

    nop()                    .side(1)   [7] # Assert MAB. 8 cycles nop and 8 cycles stop-bits = 16us

    # Send data frame
    wrap_target()
    
    pull()                   .side(1)   [7] # 2 STOP bits,  1 + 7 clocks,   or stall with line in idle state (extending MAB) 
    set(x, 7)                .side(0)   [3] # 1 START BIT  1 + 4 clocks load bit counter, assert start bit for 4 clocks

    label("bitloop")
    out(pins, 1)                        [2]  # Shift 1 bit from OSR to the first OUT pin
    jmp(x_dec, "bitloop")    .side(0)        # Each loop iteration is 4 cycles.

    wrap()
# fmt: on


# Usage example
sm_dmx_tx = StateMachine(
    1,
    dmx_send,
    freq=1_000_000,
    set_base=pin_dmx_tx,
    out_base=pin_dmx_tx,
    sideset_base=pin_dmx_tx,
)
sm_dmx_tx.active(1)

In [180]:
# start capture
# setup logic capture
from saleae import automation

manager = automation.Manager.connect()
# setup logic capture on all channels
device_configuration = automation.LogicDeviceConfiguration(
    enabled_digital_channels=[0, 1, 2, 3, 4, 5, 6, 7],
    digital_sample_rate=12_000_000,
)
capture = manager.start_capture(device_configuration=device_configuration)

In [181]:
# %%micropython
p1.on()
import time
size = 512
from array import array

universe = array("B", [0] + [255] * (size))  # 1 start code + 512 channels

# print(f"{len(universe)=} {universe=}")

for i in range(len(universe)):
    universe[i] = i % 256

universe[0] = 123 # test Start Code

for n in range(2):
    sig_dmx_send.on()
    sm_dmx_tx.restart()
    sm_dmx_tx.active(1)
    if sm_dmx_tx.tx_fifo():
        print("TX FIFO not empty before put")
    sm_dmx_tx.put(universe)
    sig_dmx_send.off()
    time.sleep(0.1)

p1.off()

In [182]:
# End capture
capture.stop()
# "Serial", "Accept DMX-1986 4us MAB"
# Serial = input channel
capture.add_analyzer("DMX-512", settings={"Serial": 5, "Accept DMX-1986 4us MAB": True})
# capture.add_analyzer(
#     "DMX-512", settings={"Serial": 3, "Accept DMX-1986 4us MAB": True}, label="Tx-Shifter"
# )
# capture.add_analyzer(
#     "DMX-512", settings={"Serial": 6, "Accept DMX-1986 4us MAB": True}, label="Tx-Shifter-2"
# )
capture.add_analyzer("DMX-512-RX", settings={"Serial": 4, "Accept DMX-1986 4us MAB": True})
# "Input Channel", "Bit Rate (Bits/s)", "Bits per Frame", "Stop Bits", "Parity Bit", "Significant Bit", "Signal inversion", "Mode"
# capture.add_analyzer("Async Serial", settings={"Input Channel": 6, "Bit Rate (Bits/s)": 115200})

InternalServerError: Analyzer not initialized